In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

The "x" of the dataset contains a two dimension numpy array of uint8, where each
row contains a 32x32 coloured image. The picture follows the "channel first" rule.

The "y" of the dataset contains a one dimension numpy array of uint8, where each
value indicates the label of corresponding x item.


In [2]:
# Load the recycled dataset
train_data = np.load('./dataset/recycled_32_train.npz')
test_data = np.load('./dataset/recycled_32_test.npz')

# split data and labels
x_train = train_data['x']
y_train = train_data['y']
x_test = test_data['x']
y_test = test_data['y']

# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0
The "x" of the dataset contains a two dimension numpy array of uint8, where each
row contains a 32x32 coloured image. The picture follows the "channel first" rule.

The "y" of the dataset contains a one dimension numpy array of uint8, where each
value indicates the label of corresponding x item.

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)

x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

We use default.qubit device to simulate the quantum circuit
Q

In [3]:
n_epochs = 50   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 10000 # Size of training dataset
n_test = 1500   # Size of testing dataset
n_qubits = 4    # Number of qubits used

# use a simple state simulator of qubit-based quantum circuit architectures as device
dev = qml.device("default.qubit", wires=n_qubits)

# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, n_qubits))

@qml.qnode(dev, interface="autograd")
def circuit(phi):
    # Encoding of 4 classical input values
    for j in range(n_qubits):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(n_qubits)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]

In [4]:
def quanv(image):
    # Convolves the input image with many applications of the same quantum circuit.
    out = np.zeros((16, 16, n_qubits))

    # Loop over the coordinates of the top-left pixel of 2 by 2 squares
    for j in range(0, 32, int(n_qubits**(0.5))):
        for k in range(0, 32, int(n_qubits**(0.5))):
            # Process a squared 2 by 2 region of the image with a quantum circuit
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(n_qubits):
                out[j // 2, k // 2, c] = q_results[c]
    return out

In [5]:
SAVE_PATH = "quanvolution/" # save path

# if already preprocessed, change to True, otherwise change to False
PREPROCESS = False

if PREPROCESS:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(x_train):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(x_test):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)


# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

In [6]:
# Step 1: Compile
def MyModel():
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(5, activation="softmax"),
    ])
    
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [10]:
# Step 2: Training
import time

start = time.time()

q_model = MyModel()

q_history = q_model.fit(
    q_train_images,
    y_train,
    validation_data=(q_test_images, y_test),
    batch_size=32,
    epochs=n_epochs,
    verbose=2,
)

print(time.time() - start)

Epoch 1/50
313/313 - 0s - loss: 1.0278 - accuracy: 0.5847 - val_loss: 0.8528 - val_accuracy: 0.6393 - 302ms/epoch - 963us/step
Epoch 2/50
313/313 - 0s - loss: 0.7262 - accuracy: 0.7179 - val_loss: 0.7095 - val_accuracy: 0.7160 - 182ms/epoch - 580us/step
Epoch 3/50
313/313 - 0s - loss: 0.6218 - accuracy: 0.7616 - val_loss: 0.6949 - val_accuracy: 0.7180 - 183ms/epoch - 584us/step
Epoch 4/50
313/313 - 0s - loss: 0.5622 - accuracy: 0.7832 - val_loss: 0.6204 - val_accuracy: 0.7427 - 180ms/epoch - 576us/step
Epoch 5/50
313/313 - 0s - loss: 0.5015 - accuracy: 0.8078 - val_loss: 0.5893 - val_accuracy: 0.7580 - 180ms/epoch - 574us/step
Epoch 6/50
313/313 - 0s - loss: 0.4560 - accuracy: 0.8288 - val_loss: 0.5564 - val_accuracy: 0.7847 - 180ms/epoch - 574us/step
Epoch 7/50
313/313 - 0s - loss: 0.4260 - accuracy: 0.8384 - val_loss: 0.5315 - val_accuracy: 0.7967 - 180ms/epoch - 576us/step
Epoch 8/50
313/313 - 0s - loss: 0.3958 - accuracy: 0.8495 - val_loss: 0.5235 - val_accuracy: 0.7920 - 180ms/epo

In [11]:
# Step 3: Evaluation
loss, accuracy = q_model.evaluate(q_test_images, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

47/47 [==============================] - 0s 645us/step - loss: 0.8199 - accuracy: 0.8393
Loss: 0.8199030160903931
Accuracy: 0.8393333554267883


In [13]:
q_model.save("./model/quantum_model.h5")